In [ ]:
# RYAN SLATTERY
# CS584 - NATURAL LANGUAGE PROCESSING
# START DATE: MARCH 21, 2022
# LAST MODIFIED: MAY 06, 2022

In [ ]:
# IMPORTING LIBRARIES

import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
import spacy
import re
import torch
import torch.nn as nn
import torch.optim as optim
import sklearn
from sklearn import metrics
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# GENERATING AND SHUFFLING OUR DATA:
# Since the Kaggle Dataset is organized by class (the first half of the dataset is all class 1, and the second half is
# all class 0), I decided to shuffle the data.
data = pd.read_csv("clickbait_data.csv")
data = data.sample(frac = 1)

headlines = data['headline'].values.tolist()
labels = data['clickbait'].values.tolist()

# SEPARATING OUR TESTING/TRAINING DATA:
train_texts, valid_texts, train_labels, valid_labels = train_test_split(headlines, labels, test_size = 0.1)
train_texts, test_texts, train_labels, test_labels = train_test_split(train_texts, train_labels, test_size = 0.1)

print("THE SIZE OF THE TRAINING SET IS:", len(train_texts))
print("THE SIZE OF THE VALIDATION SET IS:", len(valid_texts))
print("THE SIZE OF THE TESTING SET IS:", len(test_texts))

In [ ]:
nlp = spacy.load('en_core_web_sm')

# FEATURE EXTRACTION:
class TF_IDF(object):
    def __init__(self, vocab_size):
        self.vocab_size = vocab_size
        self.tf = defaultdict(lambda: 0)
        self.df = defaultdict(lambda: 0)
        self.word2idx = {}
        
    # PREPROCESSING METHOD:
    # INPUT: text (a string of text)
    # OUTPUT: text (a cleaned version of the input text)
    # A simple preprocessor function that replaces uppercase letters with lowercase ones, and removes punctuation, 
    # numbers, and urls (if there are any... it is unlikely since the text only consists of headlines).
    def Preprocessor(self, text):
        text = re.sub(r'[^\w\s]', '', text)
        text = re.sub(r'http\S+', '', text)
        text = re.sub(r'[0-9]+', '', text)
        text = text.lower()

        return text
    
    # TOKENIZER METHOD:
    # INPUT: text (a string of text)
    # OUTPUT: tokens (a list of tokens)
    def Tokenizer(self, text):
        docs = nlp.tokenizer.pipe(text, batch_size = 1000)
        tokens = []

        for doc in docs:
            for token in doc:
                if token.is_stop == False:
                    tokens.append(token)

        return tokens
    
    # COUNT STOP WORDS METHOD:
    # INPUT: text (a string of text)
    # OUTPUT: len(stops) (number of stopwords within the input text)
    def CountStopWords(self, text):
        docs = nlp.tokenizer.pipe(text, batch_size = 1000)
        stops = 0
        length = 0
        
        for doc in docs:
            stops = [token for token in doc if token.is_stop == True]
            length = [token for token in doc]

        return len(stops), len(stops)/len(length)
    
    # PRONOUN SCORE METHOD:
    # INPUT: text (a string of text)
    # OUTPUT: score (number of pronouns over length of text)
    def PronounScore(self, text):
        pronouns = ["i", "you", "we'll", "he", "she", "him", "her", "you'll", "your", "his", "hers", "they", "them", "theirs", "me", "mine", "it", "yours", "we", "their", "my"]
        text = text.lower()
        text = text.split()
        
        pros = [word for word in text if word in pronouns]
        score = len(pros)/len(text)
        
        return len(pros), score
        
 
    # FIT METHOD:
    # INPUT: texts (a list of texts)
    # Calls on the Preprocessor and Tokenizer to clean and tokenize the text. Takes the resulting tokens and 
    # stores them within the tf and df dictionaries initialized in the constructor.
    def Fit(self, texts):
        num_text = 0
        self.tf['<PAD>'] = 0
        self.df['<PAD>'] = 0
        
        for text in tqdm(texts, desc = 'FITTING TEXTS'):
            clean_text = self.Preprocessor(text)
            tokens = self.Tokenizer([clean_text])
            
            for token in tokens:
                self.df[token.text_with_ws] += 1
                
                if (token.text_with_ws not in self.tf):
                    self.tf[token.text_with_ws] = [num_text]
                else:
                    self.tf[token.text_with_ws].append(num_text)
            
            num_text += 1
        
        
        if self.tf is not None:
            self.tf = {key: self.tf[key] for key in list(self.tf.keys())[:self.vocab_size]}
        
        self.word2idx = {key: idx for idx, key in enumerate(self.tf.keys())}

    
    # TRANSFORM METHOD:
    # INPUT: texts (list of texts), sequence_length (defined length of sequences)
    # OUTPUT: tf_idf (feature vectors), text_index_list (indices for each word in the vocabulary, for each doc), 
    # num_stop_words (number of stop words for each document)
    def Transform(self, texts, sequence_length):
        tf_idf = np.zeros((len(texts), self.vocab_size))
        num_stop_words = np.zeros((len(texts), 1))
        num_pronouns = np.zeros((len(texts), 1))
        stop_words_ratio = np.zeros((len(texts), 1))
        pronoun_score = np.zeros((len(texts), 1))
        text_index_list = np.zeros((len(texts), sequence_length))
        
        for i, text in tqdm(enumerate(texts), desc = 'TRANSFORMING', total = len(texts)):
            clean_text = self.Preprocessor(text)
            tokens = self.Tokenizer([clean_text])
            num_stops, stop_ratio = self.CountStopWords([clean_text])
            num_stop_words[i] = num_stops
            stop_words_ratio[i] = stop_ratio
            num_pros, pronoun_score[i] = self.PronounScore(text)
            num_pronouns[i] = num_pros
        
            doc_words = len(tokens)

            tf = 0
            idf = 0
            count = 0
            for item in tokens:
                if self.tf.get(item.text_with_ws) is not None:
                    if (count < sequence_length):
                        text_index_list[i][count] = self.word2idx[item.text_with_ws]
                        count += 1
                    tf_idf[i][self.word2idx[item.text_with_ws]] += (1/doc_words) * np.log((len(texts)/self.df[item.text_with_ws]))
        
        tf_idf = np.append(tf_idf, stop_words_ratio, axis = 1)
        tf_idf = np.append(tf_idf, pronoun_score, axis = 1)
        return tf_idf, text_index_list, num_stop_words, num_pronouns
        

In [ ]:
vocab_size = 5000
sequence_length = 10

tfidf_extractor = TF_IDF(vocab_size)
tfidf_extractor.Fit(train_texts)

train_set, train_indices, train_stops, train_pronouns = tfidf_extractor.Transform(train_texts, sequence_length)
valid_set, valid_indices, valid_stops, valid_pronouns = tfidf_extractor.Transform(valid_texts, sequence_length)
test_set, test_indices, test_stops, test_pronouns = tfidf_extractor.Transform(test_texts, sequence_length)

In [ ]:
# VISUALIZATION OF STOPWORDS BETWEEN CLICKBAIT AND NON-CLICKBAIT TEXTS
stops_0 = np.zeros((15, 1))
stops_1 = np.zeros((15, 1))

for i in range(len(train_stops)):
    if (train_labels[i] == 1):
        stops_1[int(train_stops[i])] += 1
    else:
        stops_0[int(train_stops[i])] += 1

x = np.arange(15)
plt.bar(x-0.2, stops_0.flatten(), 0.4)
plt.bar(x+0.2, stops_1.flatten(), 0.4)
plt.title('STOPWORDS FOR CLICKBAIT AND NON-CLICKBAIT TITLES')
plt.ylabel('NUMBER OF ARTICLES')
plt.xlabel('NUMBER OF STOPWORDS')
plt.legend(['NON-CLICKBAIT', 'CLICKBAIT'])

In [ ]:
# VISUALIZATION OF PRONOUNS BETWEEN CLICKBAIT AND NON-CLICKBAIT TEXTS
pros_0 = np.zeros((10, 1))
pros_1 = np.zeros((10, 1))

for i in range(len(train_pronouns)):
    if(train_labels[i] == 1):
        pros_1[int(train_pronouns[i])] += 1
    else:
        pros_0[int(train_pronouns[i])] += 1
    
x = np.arange(10)
plt.bar(x-0.2, pros_0.flatten(), 0.4)
plt.bar(x+0.2, pros_1.flatten(), 0.4)
plt.title('PRONOUNS FOR CLICKBAIT AND NON-CLICKBAIT TITLES')
plt.ylabel('NUMBER OF ARTICLES')
plt.xlabel('NUMBER OF PRONOUNS')
plt.legend(['NON-CLICKBAIT', 'CLICKBAIT'])

In [ ]:
# HYPERPARAMETERS:
embedding_size = 300
hidden_size = 300

# HYPERPARAMETERS FOR LOGISTIC REGRESSION:
num_epochs_logreg = 25
learning_rate_logreg = 0.01

# HYPERPARAMETERS FOR CNN:
num_epochs_cnn = 10
learning_rate_cnn = 0.001
num_kernels_cnn = 20

# HYPERPARAMETERS FOR CNN W/ EMBEDDINGS:
num_epochs_cnn_e = 20
learning_rate_cnn_e = 0.025
num_kernels_cnn_e = 20

In [ ]:
# BASIC LOGISTIC REGRESSION MODEL:
# Contains one output layer, with the result transformed by the sigmoid activation function.
class LogisticRegression(nn.Module):
    
    # CLASS CONSTRUCTOR:
    # INPUT: vocab_size (size of vocabulary)
    # NOTE: vocab_size + 2 for input layer due to 2 extra features that were added.
    def __init__(self, vocab_size):
        super(LogisticRegression, self).__init__()
        
        self.output_layer = nn.Linear(vocab_size + 2, 1)
        self.sigmoid = nn.Sigmoid()
    
    # FORWARD PROPOGATION METHOD:
    # INPUT: x (input data)
    # OUTPUT: y_hat (predicted values between 0 and 1)
    # Each batch of inputs get transformed by the linear output layer and the sigmoid activation.
    def forward(self, x):
        output = self.output_layer(x)
        y_hat = torch.squeeze(self.sigmoid(output))
        
        return y_hat

# CONVOLUTIONAL NEURAL NETWORK MODEL:
# Contains a hidden layer, convolution layer, and output layer from a fully-connected feedforward network.
class ClickbaitCNN(nn.Module):
    
    # CLASS CONSTRUCTOR:
    # INPUT: vocab_size (size of vocabulary), hidden_size (size of the hidden layer), num_kernels (number of kernels)
    def __init__(self, vocab_size, hidden_size, num_kernels):
        super(ClickbaitCNN, self).__init__()
        
        self.hidden_layer = nn.Linear(vocab_size + 2, hidden_size)
        self.CNN = nn.Conv1d(hidden_size, num_kernels, kernel_size = 1)
        self.pooling = nn.MaxPool1d(num_kernels)
        self.output_layer = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p = 0.2)
        
    # FORWARD PROPOGATION METHOD:
    # INPUT: x (input data)
    # OUTPUT: y_hat (predicted values between 0 and 1)
    # Each batch of data is transformed by the linear hidden layer, 1D convolution layer, and max-pooled. The processed
    # input then gets transformed by another linear output layer and the sigmoid function.
    # Slight dropout was included to prevent overfitting.
    def forward(self, x):
        x = self.dropout(torch.unsqueeze(self.hidden_layer(x), 2))
        conv = torch.squeeze(self.CNN(x), 2)
        conv = self.pooling(conv)
        output = self.output_layer(conv)
        y_hat = torch.squeeze(self.sigmoid(output), 1)
        
        return y_hat
    
# TRAINED WORD EMBEDDING CONVOLUTIONAL NEURAL NETWORK MODEL:
# Contains an embedding layer, convolution layer, and output layer from a fully-connected feedforward network.
# The convolution was done with unigrams, as larger order n-grams had issues due to short sequences.
class ClickbaitCNN_E(nn.Module):
    
    # CLASS CONSTRUCTOR:
    # INPUT: vocab_size (size of vocabulary), embedding_size (size of the embedding layer), num_kernels (number of kernels)
    def __init__(self, vocab_size, embedding_size, num_kernels, sequence_length):
        super(ClickbaitCNN_E, self).__init__()
        
        self.embedding_layer = nn.Embedding(vocab_size, embedding_size)
        self.CNN = nn.Conv1d(embedding_size, 1, kernel_size = 1)
        self.pooling = nn.MaxPool1d(sequence_length)
        self.output_layer = nn.Linear(1, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p = 0.1)
    
    # FORWARD PROPOGATION METHOD:
    # INPUT: x (input data)
    # OUTPUT: y_hat (predicted values between 0 and 1)
    # Each batch of indices is connected with the embedding layer, 1D convolution layer, and max-pooled. The processed
    # input then gets transformed by another linear output layer and the sigmoid function.
    # Slight dropout was included to prevent overfitting.
    def forward(self, x):
        e = self.dropout(torch.transpose(self.embedding_layer(x), 1, 2))
        conv = self.CNN(e)
        conv = self.pooling(conv)
        output = self.output_layer(conv)
        y_hat = torch.squeeze(torch.squeeze(self.sigmoid(output), 2), 1)
        
        return y_hat
        

In [ ]:
LogReg = LogisticRegression(vocab_size)
print(LogReg)

CNN = ClickbaitCNN(vocab_size, hidden_size, num_kernels_cnn)
print(CNN)

CNN_E = ClickbaitCNN_E(vocab_size, embedding_size, num_kernels_cnn_e, sequence_length)
print(CNN_E)

In [ ]:
# STOCHASTIC GRADIENT DESCENT TRAINING METHOD:
# INPUT: model (model type), training_data (features or embeddings), training_labels (0 and 1), validation_data,
# validation_labels, num_epochs (number of epochs **HYPERPARAMETER**), learning_rate (learning rate **HYPERPARAMETER**)
# embed (boolean value that converts embedding indices into proper type)
# For each epoch, SGD randomly selects instances from the entire training set, feeds them into the model, and calculates
# the loss from the output. Binary Cross Entropy Loss is used in this case.
# Validation data and labels are passed in as a full batch and are evaluated at the end of each epoch.
def SGD(model, training_data, training_labels, validation_data, validation_labels, num_epochs, learning_rate, embed = False):
    train_losses = []
    valid_losses = []
    
    model_optimizer = optim.SGD(model.parameters(), lr = learning_rate)
    criterion = nn.BCELoss()
    
    for i in tqdm(range(num_epochs), desc = "TRAINING MODEL"):
        total_train_loss = 0
        for j in range(len(training_data)):
            rand = np.random.randint(0, len(training_data))
            
            x = torch.unsqueeze(torch.tensor(training_data[rand], dtype = torch.float), 0)
            if (embed == True):
                x = torch.unsqueeze(torch.tensor(training_data[rand], dtype = torch.long), 0)
                
            y = torch.tensor(training_labels[rand], dtype = torch.float)
            
            y_hat = model(x)
            train_loss = criterion(torch.squeeze(y_hat), y)
            total_train_loss += train_loss
            
            train_loss.backward()
            model_optimizer.step()
            model_optimizer.zero_grad()
            
        
        print("TRAINING LOSS AT EPOCH", i + 1, "IS:", float(total_train_loss / len(training_data)))
        train_losses.append(float(total_train_loss / len(training_data)))
        
        valid_loss = 0
        valid_x = torch.tensor(validation_data, dtype = torch.float)
        if (embed == True):
            valid_x = torch.tensor(validation_data, dtype = torch.long)
        valid_y = torch.tensor(valid_labels, dtype = torch.float)
        valid_y_hat = model(valid_x)
        valid_loss = criterion(valid_y_hat, valid_y)
        
        print("VALIDATION LOSS AT EPOCH", i + 1, "IS:", float(valid_loss), '\n')
        valid_losses.append(float(valid_loss))
        
    
    return train_losses, valid_losses
            
            

In [ ]:
# TRAINING THE LOGISTIC REGRESSION MODEL:
logreg_train_losses, logreg_valid_losses = SGD(LogReg, train_set, train_labels, valid_set, valid_labels, num_epochs_logreg, learning_rate_logreg, False)

In [ ]:
epochs = []

for num in range(num_epochs_logreg):
    epochs.append(num)

plt.plot(epochs, logreg_train_losses)
plt.plot(logreg_valid_losses)
plt.legend(['TRAINING LOSSES', 'VALIDATION LOSSES'])
plt.title("LOSS PER EPOCH")

In [ ]:
# EVALUATING THE MODEL WITH THE TEST SET
# INPUT: model (type of model), test_data (testing data (features or embeddings)), test_labels (0 and 1)
# model_name (name of model), embed (boolean value that converts embedding indices into proper type)
def Test(model, test_data, test_labels, model_name, embed = False):
    x = torch.tensor(test_data, dtype = torch.float)
    if (embed == True):
        x = torch.tensor(test_data, dtype = torch.long)
    y_hat = model(x)
    y_hat = list([int(i.round()) for i in y_hat])

    accuracy = metrics.accuracy_score(test_labels, y_hat)
    precision = metrics.precision_score(test_labels, y_hat)
    recall = metrics.recall_score(test_labels, y_hat)
    confusion_matrix = metrics.confusion_matrix(test_labels, y_hat)
    print("ACCURACY SCORE FOR", model_name, "IS:", accuracy)
    print("PRECISION SCORE FOR", model_name, "IS:", precision)
    print("RECALL SCORE FOR", model_name, "IS:", recall, '\n')
    print("CONFUSION MATRIX:", '\n', confusion_matrix)

In [ ]:
Test(LogReg, test_set, test_labels, "LOGISTIC REGRESSION")

In [ ]:
# TRAINING CNN WITH TF-IDF AND ENGINEERED FEATURES
cnn_train_losses, cnn_valid_losses = SGD(CNN, train_set, train_labels, valid_set, valid_labels, num_epochs_cnn, learning_rate_cnn, False)

In [ ]:
epochs = []

for num in range(num_epochs_cnn):
    epochs.append(num)

plt.plot(epochs, cnn_train_losses)
plt.plot(cnn_valid_losses)
plt.legend(['TRAINING LOSSES', 'VALIDATION LOSSES'])
plt.title("LOSS PER EPOCH")

In [ ]:
Test(CNN, test_set, test_labels, "CNN")

In [ ]:
# TRAINING CNN WITH WORD EMBEDDINGS:
cnn_e_train_losses, cnn_e_valid_losses = SGD(CNN_E, train_indices, train_labels, valid_indices, valid_labels, num_epochs_cnn_e, learning_rate_cnn_e, True)

In [ ]:
epochs = []

for num in range(num_epochs_cnn_e):
    epochs.append(num)

plt.plot(epochs, cnn_e_train_losses)
plt.plot(cnn_e_valid_losses)
plt.legend(['TRAINING LOSSES', 'VALIDATION LOSSES'])
plt.title("LOSS PER EPOCH")

In [ ]:
Test(CNN_E, test_indices, test_labels, "CNN WITH EMBEDDINGS", True)